In [1]:
import numpy as np
import time
np.random.seed(10)
import os
import shutil
from sklearn.metrics import accuracy_score

In [2]:
import cv2
# img = cv2.imread('cat/cat0.jpg', -1) # -1 for unchanged, 0 for grayscale and 1 for color
# img2 = cv2.imread('cat/cat2.jpg', -1)
# img = img.reshape((1, 200, 200, 3))
# img2 = img2.reshape((1, 200, 200, 3))
# # cv2.imwrite('cat.jpg',img)
# img = np.concatenate((img, img2), axis = 0)

In [4]:
# Resize all images
# X = np.array([], dtype=np.int64).reshape(0, 224, 224, 3)

# dim = (224, 224)
# i = 0
# for file in os.listdir('train'):
#     img = cv2.imread(os.path.join('train', file), -1)
#     resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
#     os.remove('train/'+file)
#     cv2.imwrite(os.path.join('train', file),resized)
#     i+=1
#     if(i%1000 == 0):
#         print(i)

In [4]:
# Randomly select cat and dogs in training set and obtain test set

# Example: shutil.move("cat/cat1.jpg", "test/cat1.jpg")

# i = 0
# for file in os.listdir('cat'):
#     r = np.random.rand()
#     if r < 0.20:
#         i+=1
#         shutil.move(os.path.join('cat', file), os.path.join('test', file))
# j = 0
# for file in os.listdir('dog'):
#     r = np.random.rand()
#     if r < 0.20:
#         j+=1
#         shutil.move(os.path.join('dog', file), os.path.join('test', file))
# print('i:'+str(i))
# print('j:'+str(j))

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

In [1]:
# Make X and y


# i = 0

# img_list = list()
# y = list()

# for file in os.listdir('train'):
#     img = img_to_array(load_img('train/' + file, target_size=(224, 224)))
#     img_list.append(img)
#     if file.startswith('cat'):
#         y.append(0)
#     else:
#         y.append(1)
#     i+=1
#     if(i%1000 == 0):
#         print(i)

# X = np.asarray(img_list)
# del img_list
# y = np.asarray(y)

In [7]:
# randomize X and y, store in npz file

# from sklearn.utils import shuffle

# y = np.concatenate((np.ones(y.shape)-y, y), axis=1)

# X, y = shuffle(X, y)
# np.savez_compressed('train/X', arr=X)
# np.savez_compressed('train/y', arr=y)

# X = np.load('train/X.npz')
# X = X['arr']
# y = np.load('train/y.npz')
# y = y['arr']
# X /= np.max(X)

In [53]:
# Place in corresponding dir

# for file in os.listdir('train'):
#     if file.startswith('cat'):
#         shutil.move('train/'+file, 'cat/'+file)
#     elif file.startswith('dog'):
#         shutil.move('train/'+file, 'dog/'+file)

# for file in os.listdir('train/cat'):
#     r = np.random.rand()
#     if r < 0.20:
#         shutil.move('train/cat/'+file, 'test/cat/'+file)
# for file in os.listdir('train/dog'):
#     r = np.random.rand()
#     if r < 0.20:
#         shutil.move('train/dog/'+file, 'test/dog/'+file)

In [4]:
import matplotlib.pyplot as plt
# index = 1197
# print(y[index])
# plt.imshow(X[index].reshape(200, 200, 3), cmap=plt.cm.binary)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

model = VGG16(include_top=False, input_shape=(224, 224, 3))

In [7]:
datagen = ImageDataGenerator(featurewise_center=True)
datagen.mean = [123.68, 116.779, 103.939]
train_generator = datagen.flow_from_directory('train', class_mode='binary', batch_size=64, target_size=(224, 224))
test_generator = datagen.flow_from_directory('test', class_mode='binary', batch_size=64, target_size=(224, 224))

Found 19961 images belonging to 2 classes.
Found 5039 images belonging to 2 classes.


In [8]:
from tensorflow.keras.models import Model
for layer in model.layers:
    layer.trainable = False
flat = Flatten()(model.output)
dense1 = Dense(128, activation = 'relu')(flat)
drop = Dropout(0.3)(dense1)
dense2 = Dense(1, activation = 'sigmoid')(drop)
model = Model(inputs=model.input, outputs=dense2)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
history = model.fit_generator(train_generator, steps_per_epoch=len(train_generator), \
                              validation_data=test_generator, validation_steps=len(test_generator), epochs=1)

312/312 [==============================] - 21470s 69s/step - loss: 0.8224 - accuracy: 0.9420 - val_loss: 0.6109 - val_accuracy: 0.9577


In [18]:
model.save('acc9420val9577')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: acc9420val9577\assets


In [33]:
# model = Sequential()
# model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform', \
#                  input_shape=(224,224,3)))
# model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(1024, activation = 'relu'))
# model.add(Dense(1024, activation = 'relu'))
# model.add(Dense(1, activation = 'softmax'))

In [24]:
# del X
# del y

# # Make testy
# testy = np.concatenate((np.zeros((2644, 1)), np.ones((2527, 1))), axis=0)
# testy.shape

# # Make testX
# img_list = list()
# for file in os.listdir('test'):
#     img = img_to_array(load_img('test/' + file, target_size=(200, 200)))
#     img_list.append(img)

# testX = np.asarray(img_list)
# np.savez_compressed('testX', arr=testX)
# del img_list
# del img

# testX = np.load('testX.npz')
# testX = testX['arr']

In [26]:
# from sklearn.utils import shuffle

# testX, testy = shuffle(testX, testy)
# testy = np.concatenate((np.ones(testy.shape)-testy, testy), axis=1)

# np.savez_compressed('testX', arr=testX)
# np.savez_compressed('testy', arr=testy)

testX = np.load('testX.npz')
testX = testX['arr']
testy = np.load('testy.npz')
testy = testy['arr']

In [1]:
accuracy_score(np.argmax(model.predict(testX), axis=1), np.argmax(testy, axis=1))

NameError: name 'accuracy_score' is not defined